# Interactive bytemaker - no local installation necessary!
Welcome to the bytemaker Jupyter notebook (or the noninteractive preview, if the binder environment is still loading)!

For full documentation and a more comprehensive guide, please check out the [Bytemaker ReadTheDocs](https://bytemaker.readthedocs.io/en/latest/index.html)

In [1]:
!pip install bytemaker
from bytemaker.bitvector import BitVector
from bytemaker.bittypes import Buffer, SInt, SInt8, SInt32, UInt3, UInt3, UInt2, Float, Float16, Float32, Str8, Str64

## 1 BitVectors
`BitVector`s are like `bytearray`s, but for bits! They support all the operations you'd expect, plus a few extra ones.
### 1.1 Constructors

In [2]:
# Default constructor
bv_empty = BitVector()
bv_empty

BitVector('')

In [3]:
# Other constructors
bv = BitVector([0, 1, 1, 0, 1])
bv2 = BitVector("0xFF")
bv3 = BitVector(b"Hello, World!")
bv4 = BitVector(5)
(bv, bv2, bv3, bv4)

(BitVector('01101'),
 BitVector('11111111'),
 BitVector('01001000 01100101 01101100 01101100 01101111 00101100 00100000 01010111 01101111 01110010 01101100 01100100 00100001'),
 BitVector('00000'))

### 1.2 Conversions

In [4]:
# Presentation/conversion
the_bitvector = BitVector(bytes("Hello, Worlds!", "utf-16"))
print(bytes(the_bitvector))
# print(the_bitvector.tobase(4, sep="_", bytes_per_sep=2)) # Also available: explicit bin, oct, hex methods
# print([i for i in the_bitvector])
# print(the_bitvector.to_chararray(encoding="utf-16"))

b'\xff\xfeH\x00e\x00l\x00l\x00o\x00,\x00 \x00W\x00o\x00r\x00l\x00d\x00s\x00!\x00'


### 1.3 Operations

In [5]:
# Modification
# ~, &, |, ^, <<, >>, ==, !=, <, >, bitvector[], etc.
bv = BitVector("0xF0")
(bv + bv << 2)[range(16)][:]

BitVector('11000011 11000000')

## 2 BitTypes
BitTypes simulate C data types (short, long, unsigned short, etc.), and give you the opportunity to define your own (if you want an exotic floating-point number, you've come to the right place!). They're internally backed by a Python-typed `value` field and a corresponding `BitVector`-typed `bits` property.

### 2.1 Construction

In [6]:
sint8_1 = SInt8(5)
sint8_2 = SInt8(value=5)
sint8_3 = SInt8(BitVector("00000101"))
sint8_4 = SInt8(bits=BitVector("00000101"))
uint8_1 = UInt3(sint8_1)
print((str(sint8_1), str(sint8_2), str(sint8_3), str(sint8_4), str(uint8_1)))
print(sint8_1 == sint8_2 == sint8_3 == sint8_4 == uint8_1)

('SInt8[big](5 = 00000101)', 'SInt8[big](5 = 00000101)', 'SInt8[big](5 = 00000101)', 'SInt8[big](5 = 00000101)', 'UInt3[big](5 = 101)')
True


### 2.2 Conversions

In [7]:
sint8_1 = SInt8(-5)
(sint8_1.bits, sint8_1.value)

(BitVector('11111011'), -5)

In [8]:
bytes(sint8_1)

b'\xfb'

### 2.3 Operations

In [9]:
sint8_1 = SInt8(-5)
sint8_2 = SInt8(bits=BitVector("00000101"))
sint_added = sint8_1 + sint8_2
str(sint_added)

'SInt8[big](0 = 00000000)'

In [10]:
str(~sint_added)

'SInt8[big](-1 = 11111111)'

In [11]:
SInt8(-1) << 3

SInt8(bits=BitVector('11111000'), endianness=big)

### 2.4 Additional Type Creation

In [12]:
SInt17 = SInt.specialize(num_bits_=17, name_="SInt17")
str(SInt17(-5))

'SInt17[big](-5 = 11111111...11111011)'

In [13]:
Bfloat16 = Float.specialize(num_exponent_bits_ = 8, num_mantissa_bits_ = 7, name_="Bfloat16")
bfloat16_1 = Bfloat16(55555.0) # An estimation is forced here. 55555.0 is not exactly representable in Bfloat16 format
(str(bfloat16_1), bfloat16_1.value)

('Bfloat16[big](55552.0 = 01000111 01011001)', 55552.0)

## 3 BitFields
bytemaker simulates C bitfield functionality by allowing you to combine bittypes with the dataclass decorator. Simply declare what types you expect, and convert away!

### 3.1 Declaration and construction
To declare a bytemaker bitfield, simply declare the members of a dataclass with your choice of python types, ctypes, or BitTypes. These can be mixed at will, and the default size of the pure-Python types can be changed.

bytemaker bitfields can be nested as many times as wished.

In [14]:
import ctypes
from bytemaker.conversions.aggregate_types import to_bits_aggregate, from_bits_aggregate
from dataclasses import dataclass

Buffer4 = Buffer.specialize(4, name_="Buffer4") # Included for illustration. Buffer4 is also provided as an import.

@dataclass
class PyTypeAggregate:
    a: int  # by default, 32-bit
    b: float# by default, 32-bit
    c: str  # Char
    d: str  # Char


@dataclass
class CTypeAggregate:
    a: ctypes.c_int32
    b: ctypes.c_float
    c: ctypes.c_char
    d: ctypes.c_char


@dataclass
class BitTypeAggregate:
    a: SInt32
    b: Float32
    c: Str8
    d: Str8
    e: Buffer4


@dataclass
class MixedAggregate:
    pytype_aggregate: PyTypeAggregate
    ctype_aggregate: CTypeAggregate
    bittype_aggregate: BitTypeAggregate
    hp: int


mixedagg = MixedAggregate(
    PyTypeAggregate(
        512,
        3.14,
        'A',
        'B'
    ),
    CTypeAggregate(
        ctypes.c_int32(512),
        ctypes.c_float(3.14),
        ctypes.c_char('A'.encode('utf-8')),
        ctypes.c_char('B'.encode('utf-8'))
    ),
    BitTypeAggregate(
        SInt32(382),
        Float32(3.14),
        Str8('A'),
        Str8('B'),
        Buffer4('0b0101')
    ),
    255
)

mixedagg

MixedAggregate(pytype_aggregate=PyTypeAggregate(a=512, b=3.14, c='A', d='B'), ctype_aggregate=CTypeAggregate(a=c_int(512), b=c_float(3.140000104904175), c=c_char(b'A'), d=c_char(b'B')), bittype_aggregate=BitTypeAggregate(a=SInt32(bits=BitVector('00000000 00000000 00000001 01111110'), endianness=big), b=Float32(bits=BitVector('01000000 01001000 11110101 11000011'), endianness=big), c=Str8(bits=BitVector('01000001'), endianness=big), d=Str8(bits=BitVector('01000010'), endianness=big), e=Buffer4(bits=BitVector('0101'), endianness=big)), hp=255)

### 3.2 Conversions and manipulations
Members of bytemaker bitfields can be freely modified and converted to/from binary representations
#### Converting to hex

In [15]:
mixed_agg_as_bits = to_bits_aggregate(mixedagg)

mixed_agg_as_bits.hex(bytes_per_sep=2, sep='_')

'0x0000_0200_4048_f5c3_4142_0000_0200_4048_f5c3_4142_0000_017e_4048_f5c3_4142_5000_000f_f'

#### Adjusting `mixedagg.bittype_aggregate.a` and converting to hex

In [16]:
mixedagg.bittype_aggregate.a = SInt32(512)
mixed_agg_as_bits = to_bits_aggregate(mixedagg)
mixed_agg_as_bits.hex(bytes_per_sep=2, sep='_')

'0x0000_0200_4048_f5c3_4142_0000_0200_4048_f5c3_4142_0000_0200_4048_f5c3_4142_5000_000f_f'

#### Reading back from hex

In [17]:
mixed_agg_from_bits = from_bits_aggregate(mixed_agg_as_bits, MixedAggregate)
mixed_agg_from_bits

MixedAggregate(pytype_aggregate=PyTypeAggregate(a=512, b=3.140000104904175, c='A', d='B'), ctype_aggregate=CTypeAggregate(a=c_int(512), b=c_float(3.140000104904175), c=c_char(b'A'), d=c_char(b'B')), bittype_aggregate=BitTypeAggregate(a=SInt32(bits=BitVector('00000000 00000000 00000010 00000000'), endianness=big), b=Float32(bits=BitVector('01000000 01001000 11110101 11000011'), endianness=big), c=Str8(bits=BitVector('01000001'), endianness=big), d=Str8(bits=BitVector('01000010'), endianness=big), e=Buffer4(bits=BitVector('0101'), endianness=big)), hp=255)